<a href="https://colab.research.google.com/github/juergenlandauer/FoundationModelsArchaeology/blob/main/Experiment_1_castles_satellite/GPT_Castles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic site detection in satellite and LiDAR images with Gemini by Google

Author: Juergen Landauer (juergen AT landauer-ai.de)

To start, first go to the "Input parameters" section below and review or (optionally) adjust parameters. Then run the entire Notebook by choosing Runtime->Run all in the menu above.


### Set up your API key and install the Open AI Python SDK

To access GPT-4, you need to provide your OpenAI API key. Follow these steps:

- register with Open AI
- Open your [`OpenAI Settings`](https://platform.openai.com/settings) page. Click `User API keys` then `Create new secret key` to generate new token.
Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑).
- Store OpenAI API key under the name `OPENAI_API_KEY`.

In [ ]:
!pip install -Uq openai
!pip install -Uq rvt-py rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/755.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

# Input parameters

Review all parameters in this section and (optionally) adjust them on the right side. For example, you can upload your own input zip file by providing an URL.

#### Demo with castles from Bavaria (Germany) in Satellite imagery from Microsoft Bing
Feel free to replace this with your own imagery by providing a download URL (e.g. from Google Drive)

Note that the ZIP file must contain two sub-folders called "sites" and "nonsites", resp. Each folder must then contain images of sites or samples of other landscape (non-sites)

In [ ]:
INPUT_ZIP_URL_NONSITES = 'https://www.dropbox.com/scl/fi/8px493jr03mgi0660tsfv/NonCastlesBingImages_768_19_V2.zip?rlkey=lr89m75x40o7sg6iikje0b2q3&st=3geldkd5&dl=0'
INPUT_ZIP_URL_SITES    = 'https://www.dropbox.com/scl/fi/vm0ya2ddb0ly8eyrsqws0/BingImages_768_19_orig.zip?rlkey=8t2gte5ssiyus3svmdxxnko0y&dl=0'

Temples

In [ ]:
INPUT_ZIP_URL_NONSITES = 'https://www.dropbox.com/scl/fi/q0mddolngvou51vyojhii/NonsitesKhmer.zip?rlkey=d718q7vonuh1g07yhajn4a8h0&dl=0'
INPUT_ZIP_URL_SITES    = 'https://www.dropbox.com/scl/fi/qhrby4cmb3rtgpptplbrl/TemplesKhmerTIF.zip?rlkey=0xxj479p68hoqxe8rlpsxvh8k&dl=0'

#### The text 'prompt' sent to the Foundation Model.

Play with different variations of the text and don't forget to include the object type you are looking for.

In [ ]:
# Castles
PROMPT = """
You are analyzing a satellite image that may contain archaeological features from Germany, such as castles, ruins, or other ancient man-made structures.

Important Note: The majority of images will likely contain no archaeological features at all.
Therefore, avoid making detections unless there is strong, high-confidence evidence above 75 percent.

For each distinct object or feature only if confidently detected, return the following in JSON format:

Object Type — classify the object (e.g., enclosure, hillfort, natural formation).

Confidence Score — your estimated probability (%) that the classification is correct.

Bounding Box — provide coordinates in the format [x_min, y_min, x_max, y_max]. Make sure the bounding box tightly encompasses the object.

Reason - textually explain why you think the given object is found.

If no archaeological features are confidently detected, return an empty list

"""

In [ ]:
# Temples
PROMPT = """
You are analyzing a satellite image that may contain archaeological features from Cambodia, such as ancient Buddhist temples or other ancient man-made structures.

For each distinct object or feature, return the following in JSON format:

Object Type — classify the object (e.g., temple, reservoir, moat, unknown).

Confidence Score — your estimated probability (%) that the classification is correct.

Bounding Box — provide coordinates in the format [x_min, y_min, x_max, y_max]. Make sure the bounding box tightly encompasses the object.

Reason - textually explain why you think the given object is found.

"""

The text response from the AI is sometimes ambiguous. If you only (or "strictly") want to see certain types of responses, then keep this to True and provide a list of keywords you want to see in the output. Make sure you do not mess up the list syntax.

In [ ]:
STRICT_RESPONSE_FILTERING = True # @param ['True', 'False']

FILTER_KEYWORDS = ["castle", "ruin", "enclosure", "hillfort"]  # @param {"allow-input":true}
FILTER_KEYWORDS = ["temple", "reservoir", "moat"]  # @param {"allow-input":true}

Here we define the model we are using. Usually it is not required to change this.

In [ ]:
MODEL = "gpt-4.1-mini" # @param ["gpt-4.1-mini","gpt-4.1-nano", "gpt-4.1"] {"allow-input":true, isTemplate: true}

In case you encounter performance issues, consider setting RESIZE to True, as it halves each image dimension

In [ ]:
RESIZE = False # @param ['True', 'False']

## We load the data and unzip it into the directory 'input'

In [ ]:
!rm -rf input output file.zip
!mkdir -p input/nonsites input/sites
!wget -O file.zip "$INPUT_ZIP_URL_NONSITES"
!unzip -q file.zip -d input/nonsites
!wget -O file.zip "$INPUT_ZIP_URL_SITES"
!unzip -q file.zip -d input/sites

--2025-07-09 09:32:21--  https://www.dropbox.com/scl/fi/q0mddolngvou51vyojhii/NonsitesKhmer.zip?rlkey=d718q7vonuh1g07yhajn4a8h0&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9880186f9d67b7e716638707d6.dl.dropboxusercontent.com/cd/0/inline/CtJ2RokE88UYeAj3nn5h7BXylj4vJP0xNa8Kv50JbsGExQM-8TbjkLUfqwgB45RsddjDpfwkR3JTyRY4So9zSTpE85aIuadpazM1JFuYkiEQuvV2frBXfHtGiAj59AP6I8o/file# [following]
--2025-07-09 09:32:22--  https://uc9880186f9d67b7e716638707d6.dl.dropboxusercontent.com/cd/0/inline/CtJ2RokE88UYeAj3nn5h7BXylj4vJP0xNa8Kv50JbsGExQM-8TbjkLUfqwgB45RsddjDpfwkR3JTyRY4So9zSTpE85aIuadpazM1JFuYkiEQuvV2frBXfHtGiAj59AP6I8o/file
Resolving uc9880186f9d67b7e716638707d6.dl.dropboxusercontent.com (uc9880186f9d67b7e716638707d6.dl.dropboxusercontent.com)... 162.125.80.15, 2620:100:6035:15::a27d:550f
Connec

### we now import some libraries

In [ ]:
import numpy as np
import os
from osgeo import gdal
import cv2 as cv
import PIL.Image
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
from time import sleep
from tqdm import tqdm
from google.colab import files as colabfiles
import json
import random
import io
from PIL import Image, ImageDraw, ImageColor, ImageFont
import re

### Some utility functions we use

In [ ]:
# remove all pixels below -10 and above 1000-min
def remove_outliers(image:np.ndarray):
    if np.isnan(image).sum() > 0:
        print ("NAN!", np.isnan(image).sum() )
    min_zero = image[image>-10].min()
    image[image<=-10] = min_zero
    # do we still have extremal values?
    mymax = np.max(image)
    mymin = np.min(image)
    if mymax-mymin>1000:
        print ("outlier removal:", mymax)
        hi = np.percentile(image, 99).min()
        image[image>1000+mymin] = hi
    return image


In [ ]:
# Function to encode the image
import base64 # Import the base64 module
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
import rvt.vis
def preprocessSnippetHillshade(image: np.array):
    image = remove_outliers(image)

    pixels = rvt.vis.hillshade(
        dem=np.squeeze(image), # remove axis...
                sun_azimuth=315,
                sun_elevation=45,
                resolution_x=1.,
                resolution_y=1.,
                ve_factor=3, # was 2 !!!!!!
                #no_data=dem_no_data
            ) # output is float32
    #pixels = np.nan_to_num(pixels, copy=True, nan=0.0, posinf=None, neginf=None)# remove NaN
    pixels = pixels[np.newaxis,...] # and add axis again

    return pixels

In [ ]:
from osgeo import gdal
#import rvt.default
import rvt.vis

def preprocessSnippetSLRM(image: np.ndarray):
    image = remove_outliers(image)
    image = rvt.vis.slrm(dem=np.squeeze(image), # remove axis...
                radius_cell=10,
                ve_factor=1,
                no_data=None
    )
    #image = image[np.newaxis,...] # and add axis again
    mymin, mymax = image.min(), image.max()
    image = np.interp(image, (mymin, mymax), (0, 255)).astype(np.uint8)
    return image

In [ ]:
import rasterio as rio
import PIL
def read_hillshade(fpath):
  with rio.open(fpath) as t:
     img = t.read().squeeze()
  img = (preprocessSnippetHillshade(img)*256).astype(np.uint8)
  #img = (preprocessSnippetSLRM(img)).astype(np.uint8)
  pilimg = PIL.Image.fromarray(img.squeeze()).convert('RGB')
  return pilimg

In [ ]:
# function to read the images as PIL
def read_pil(fpath):
  pilimg = PIL.Image.open(fpath)
  return pilimg

In [ ]:
# function to plot bounding boxes on images
def plot_bounding_boxes(img, results):
    width, height = img.size
    # Create a drawing object
    draw = ImageDraw.Draw(img)
    fpath='/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf'
    fontsize = 24
    font = ImageFont.truetype(fpath, fontsize)

    noun_phrase, prob, bbox = results

    if len(bbox) != 4: return # nothing found
    x1, y1, x2, y2 = bbox
    # Ensure x1 <= x2 and y1 <= y2
    x1, x2 = sorted([x1, x2])  # Sort x-coordinates
    y1, y2 = sorted([y1, y2])  # Sort y-coordinates

    color = 'yellow'
    # Draw the bounding box
    draw.rectangle(((x1, y1), (x2, y2)), outline=color, width=4)
    # Draw the text
    draw.text((x1 + 8, y1 + 6), noun_phrase+" "+str(prob), fill=color, font=font)

# process folder

In [ ]:
sites = sorted(glob('./input/sites/**/*.*', recursive=True))
nonsites = sorted(glob('./input/nonsites/**/*.*', recursive=True))
len(sites), len(nonsites)

(100, 1000)

In [ ]:
# uncomment this if you want to try just a small sample of MAX_N for each class
import random
MAX_Sites = 20
MAX_Nonsites = 40

#sites = random.sample(sites, MAX_Sites)
#nonsites = random.sample(nonsites, MAX_Nonsites)

#sites = sites[0:MAX_Sites]
#nonsites = nonsites[0:MAX_Nonsites]

In [ ]:
len(sites), len(nonsites)

(100, 1000)

In [ ]:
# define the Gemini output format of a 'Detection'
from pydantic import BaseModel, TypeAdapter
class Detection(BaseModel):
  detection_type: str
  probability: float
  bbox: list[int]
  reason: str

In [ ]:
#visualizeFN = read_hillshade
visualizeFN = read_pil

## Processing all files with GPT

In [ ]:
!rm -rf output
!mkdir output output/FN output/TP output/FP output/TN

In [ ]:
# configure parameters
#TEMPERATURE = 1.0
#TEMPERATURE = 0.7
TEMPERATURE = 0.3

In [ ]:
files = sites + nonsites
#files = nonsites

In [ ]:
files = sites + nonsites

with open('./output/phrases.txt', 'w') as docfile:
  TP, FP, FN, TN = 0, 0, 0, 0
  for fpath in tqdm(files):
    img = read_pil(fpath)
    new_size = (img.width // 2, img.height // 2)
    img2 = img.copy().resize(new_size) if RESIZE else img

    img2.save('./tmp.png')
    base64_image = encode_image("./tmp.png")
    print('________________________________________________________')

    response_format = Detection
    for attempt in range(10):
      try:
        response = client.responses.create(
          model = MODEL,
          temperature = 0.3,
          input=[{
             "role": "user",
            "content": [
                {"type": "input_text", "text": PROMPT},
                {"type": "input_image", "image_url": f"data:image/jpeg;base64,{base64_image}",
          },],}],
          text={
            "format": {
                "type": "json_schema",
                "name": "Detection",
              "schema": {
                  "type": "object",
                  "properties": {
                      "detection_type": {
                          "type": "string"},
                      "probability": {
                          "type": "number"
                      },
                      "bbox": {
                          "type": "array",
                          "items": {
                              "type": "number"}
                      },
                      "reason": {"type": "string"},
                  },
                  "required": ["detection_type", "probability", "bbox", "reason"],
                  "additionalProperties": False
              },
              "strict":True
            },},)
        if response.output_text:
          break # we got through!
        #else:
        #  print ("!no output - retry")
      except Exception as e:
        print (e);sleep(5);continue
      else:
        print ("next attempt", attempt)

    outlist = re.findall(r'({.*?})', response.output_text, re.DOTALL)

    FOUND = False
    for out in outlist:
      try:
        ret = json.loads(out)
      except json.JSONDecodeError as e:
        print(f"Could not decode JSON object: {out}")
        print(f"Error: {e}")
      except Exception as e:
        print(f"An unexpected error occurred while processing: {out}")
        print(f"Error: {e}")

      if STRICT_RESPONSE_FILTERING:
        if any(element in ret['detection_type'].lower() for element in FILTER_KEYWORDS) and float(ret['probability']) >= 0.75:
          FOUND = True
          plot_bounding_boxes(img, (ret['detection_type'],ret['probability'], ret['bbox']))
      else:
        p = ""

    if FOUND:
      if 'nonsites' in fpath: FP += 1; p="FP"
      else: TP += 1;p="TP"
    else:
      if 'nonsites' in fpath: TN += 1;p="TN"
      else: FN += 1;p="FN"

    print (fpath, "------", FOUND, "---", ret)
    display(img.resize(size=(384,384)))

    filename = Path(fpath).name
    img.save(Path('output')/Path(p)/filename)
    docfile.write("--- " + fpath + ":" + json.dumps(outlist) + os.linesep)
    print("TP, FP, FN, TN=", TP, FP, FN, TN, "(",TP+FN,")")
    sleep(1.5)

In [ ]:
F1 = 2*TP/(2*TP+FP+FN)
print (TP, FP, FN, TN)
print ("TPR=", TP/len(sites), "FPR=", FP/len(nonsites))
print ("********* F1:", F1, " *************")

with open('./output/phrases.txt', 'a') as docfile:
  docfile.write(f"---RESULTS------{os.linesep}")
  docfile.write(f"TP, FP, FN, TN={TP}, {FP}, {FN}, {TN}{os.linesep}")
  docfile.write(f"F1={F1}{os.linesep}")

57 98 43 902
TPR= 0.57 FPR= 0.098
********* F1: 0.4470588235294118  *************


## Export results for download
We now open a file download dialog for the output.zip. Simply store the output in your local computer. Done :-)

output.zip contains all images with bounding box annotations and a file phrases.txt containing the original response from Gemini.



In [ ]:
!zip -r output.zip output

  adding: output/ (stored 0%)
  adding: output/TN/ (stored 0%)
  adding: output/TN/r_839_103.76486181009244_13.968886375635527_19.tif (deflated 25%)
  adding: output/TN/r_245_103.4212428568916_12.581815211975691_19.tif (deflated 29%)
  adding: output/TN/r_921_103.9463937447268_14.332502900278795_19.tif (deflated 35%)
  adding: output/TN/r_301_104.17257172165809_12.85711576720709_19.tif (deflated 50%)
  adding: output/TN/r_924_104.78279775896907_14.323916096313589_19.tif (deflated 30%)
  adding: output/TN/r_490_104.56600088169326_14.128015013771822_19.tif (deflated 38%)
  adding: output/TN/r_59_103.1407067557081_14.29495364896564_19.tif (deflated 29%)
  adding: output/TN/r_437_102.97736789210413_13.881127080875524_19.tif (deflated 46%)
  adding: output/TN/r_349_103.38378474692854_13.830191702999658_19.tif (deflated 37%)
  adding: output/TN/r_47_103.18602067509724_12.460663626594394_19.tif (deflated 47%)
  adding: output/TN/r_306_103.91355232179389_12.492361117646222_19.tif (deflated 51%

In [ ]:
colabfiles.download('output.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>